# Test de la Chi-Cuadrado
Permite un criterio sólido para encontrar relaciones entre variables cualitativas utilizando tablas de contingencia

El fundamento de este test es comparar la suma de diferencias al cuadrado entre lo que estamos observando, es decir, nuestros datos reales, y aquello que cabría esperar en una situación donde no existe ninguna relación entre variables.

Este permite cuantificar la magnitud de estas distancias, compararlas con una distribución que es la de 'x' cuadrado, y decidir si podemos afirmar que estas variables están relacionadas significativamente.

Una de las debilidades de este test es que no permite cuantificar cada una de las relaciones entre categorías, pero permite afirmaciones globales del tipo "fumar está relacionado significativamente con el cáncer de pulmón"
- Aun así, una exploración visual de las tablas permite determinar en qué dirección se da esta relación.


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('base_datos_2008.csv')

### Fijar una semilla

In [2]:
np.random.seed(0) # Al fijar una semilla se hace que cuando quiera replicar el código, se va a obtener los mismos resultados aun cuando se apliquen distintas funciones aleatorias

# Crear un subset de los datos, seleccionar solamente aquellos casos, aquellos vuelos, que tengan un origen en uno de los tres aeropuertos seleccionado.
df = df[df['Origin'].isin(['HOU','ATL','IND'])] # Filtrar
# Reordenar la base de datos
df = df.sample(frac = 1)
# Seleccionar las primeras 10000 filas
df = df[0:10000] # Un test con millones de filas no tiene demasiado sentido, ya que siempre vamos a encontrar algun tipo de relacion

### Generar una nueva variable

In [5]:
# Esta nueva variable va a distinguir entre dos casos: Va a tener un valor 'true' cuando el 'ArrDelay' sea mas grande que 30, que implicara que tenemos un gran retraso de este vuelo, por otro lado va a tener un valor 'false' cuando sea mas pequeño o igual a 30
df['BigDelay'] = df['ArrDelay'] > 30
# Crear una tabla de contingencia, para ver que estamos haciendo concretamente. 
# Se selecciona las dos columans que se quiere estudiar, el parametro que depende es columns, por lo que se puede crear una tabla de contingencia de multiples dimensiones.
# Con margins = True, se le indica que queremos que guarde tambien los margenes de la tabla
observados = pd.crosstab(index=df['BigDelay'],columns=df['Origin'],margins=True)

observados

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,6927,883,765,8575
True,1197,129,99,1425
All,8124,1012,864,10000


El 6900 es el numero de vuelos que se han retrasado poco y que han salido de Atlanta.
Los márgenes son los elementos hasta la derecha y la fila inferior, el margen inferior es la suma por cada una de las columnas y el margen derecho es la suma para cada una de las filas.
La izquina inferior derecha es la suma de todas las observaciones.

### Importar paquete que va a realizar el test

In [7]:
from scipy.stats import chi2_contingency

In [9]:
# Pedir que cree un objeto a partir de la tabla
test = chi2_contingency(observados)
test 

(8.939538453043031,
 0.17700704816414425,
 6,
 array([[ 6966.33,   867.79,   740.88,  8575.  ],
        [ 1157.67,   144.21,   123.12,  1425.  ],
        [ 8124.  ,  1012.  ,   864.  , 10000.  ]]))

In [ ]:
 Esto nos devuelve varia información. El primero de todos es el estadístico, es decir, la suma de las diferencias al cuadrado. El segundo de ellos es el p-valor
 **El p-valor** es una medida que permite tomar decisiones basadas en algun criterio.

 La tabla es la tabla de valores esperados, que se comparara con los valores observados


### Convertir tabla en un DataFrame

In [10]:
esperados = pd.DataFrame(test[3])
esperados # Son los valores teoricos esperados si no hubiese ningun tipo de relacion
# Estos valores no son posibles ya que son decimales, es una aproximacion teorica

,0,1,2,3
0,6966.33,867.79,740.88,8575.0
1,1157.67,144.21,123.12,1425.0
2,8124.00,1012.00,864.00,10000.0


In [11]:
# Valores esperados relativos
esperados_rel = round(esperados.apply(lambda r: r/len(df) * 100, axis = 1), 2) # Redondeo para que sea mas facil de interpretar
# Se aplica una funcion lambda, es decir, una funcion sin nombre, que sera cada valor dividido entre el total de valores por 100
# Asi se genera tablas con valores relativos y no valores absolutos, que son mas dificiles de comparar

In [12]:
# Valores observados relativos
observados_rel = round(observados.apply(lambda r: r/len(df) * 100, axis = 1), 2)

In [15]:
observados_rel

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,69.27,8.83,7.65,85.75
True,11.97,1.29,0.99,14.25
All,81.24,10.12,8.64,100.00


In [16]:
esperados_rel

,0,1,2,3
0,69.66,8.68,7.41,85.75
1,11.58,1.44,1.23,14.25
2,81.24,10.12,8.64,100.00


Los margenes se deben mantener, sino significa que algo se hizo mal.

Lo que hace el test es comparar si una cantidad es suficientemente mayor al valor de la tabla observada.

El test de xi cuadrado ofrece una herramienta estadistica precisa de decision, si cierta diferencia es muy grande o muy pequeña

De hecho, lo que va a hacer el test es comparar este elemento con este, este elemento con este, este con este, etc., sumarlos, elevarlos al cuadrado, dividirlos y luego va a generar un estadístico que va a comparar con una distribución. Aquí, al final de todo, vemos un criterio de decisión. 

El **p-valor**  puede resumirse como la probabilidad de que las relaciones sean mucho más extremas de lo que hemos visto en nuestros datos si comparamos con la situación donde no hay ninguna relación. Dicho de otra forma, es un indicador de cómo de extraño sería ver nuestros datos si no hubiese relación entre variables. Un valor muy bajo, como vemos aquí, implica que hay relación significativa. Y un valor muy alto, que no podemos afirmar que la haya. Vamos a ver cuál es el resultado concreto en nuestro test y en qué situación nos encontramos.

### Resumen
- Si el p-valor < 0.05, hay diferencias significativas: Hay relacion entre variables
- Si el p-valor > 0.05, no hay diferencias significativas: No hay relacion entre variables


In [17]:
test[1] # El resultado es bastante mayor que el limite que nos fijamos por lo que no podemos determianr que hay relacion entre variables

0.17700704816414425